# [Omdena-NeedEnergy "Increasing Clean Energy Access in Africa" project](https://omdena.com/projects/clean-energy-ai/)
#### A.Montesino (Feb 10th, 2021)
### Task X - XYZ

References:
* [Document #1](#http://www.loremipzum.com/en/)
* [Document #2](#http://www.loremipzum.com/es/)
* other...


In [4]:
# Install common libraries

# NumPy
try:
    import numpy as np
    # from numpy import *
except ImportError as e:
    # module doesn't exist, deal with it.
    ! pip install numpy
print( "NumPy version:", np.__version__ )

# xlrd required by pandas' read_excel()
try:
    import xlrd 
except ImportError as e:
    # module doesn't exist, deal with it.
    ! pip install xlrd

# pandas
try:
    import pandas as pd
    from pandas import *
except ImportError as e:
    # module doesn't exist, deal with it.
    ! pip install pandas==0.25.0
print( "Pandas version:", pandas.__version__ )    

# -----------------------------------------------------------

# maptplot
try:
    import matplotlib.pyplot as plt
    #from pandas import *
except ImportError as e:
    # module doesn't exist, deal with it.
    ! pip install pandas

# Seaborn
try:
    import seaborn as sns
except ImportError as e:
    # module doesn't exist, deal with it.
    ! pip install seaborn

# Comment this if the data visualisations doesn't work on your side
%matplotlib inline
plt.style.use('bmh')

NumPy version: 1.19.5
Pandas version: 1.1.5


In [ ]:
# pip list
import numexpr
numexpr.print_versions()

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Numexpr version:   2.7.2
NumPy version:     1.19.5
Python version:    3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
Platform:          linux-x86_64-#1 SMP Thu Jul 23 08:00:38 PDT 2020
CPU vendor:        
CPU model:         
CPU clock speed:    MHz
VML available?     False
Number of threads used by default: 2 (out of 2 detected cores)
Maximum number of threads: 64
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


For details on accesing data from Google.Colab, through Google Drive, please go to: 
* [Google Colab - Local File System - API REST](https://colab.research.google.com/notebooks/io.ipynb#scrollTo=jRQ5_yMcqJiV)  
* [Importing files from Google Drive in Colab - Mounting Google Drive](https://buomsoo-kim.github.io/colab/2020/05/09/Colab-mounting-google-drive.md)  

In [35]:
# First step is getting the authorization code by loggin into your Google account. Then, paste the authorization code and press Enter.

from google.colab import drive
drive.mount('/content/drive')
!cd /content/drive/MyDrive/Data
!ls -laR 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
.:
total 1588
drwxr-xr-x 1 root root    4096 Feb 22 23:30 .
drwxr-xr-x 1 root root    4096 Feb 22 22:36 ..
drwxr-xr-x 1 root root    4096 Feb 16 16:35 .config
drwx------ 5 root root    4096 Feb 22 23:30 drive
drwxr-xr-x 1 root root    4096 Feb 16 16:35 sample_data
-rw-r--r-- 1 root root 1602868 Feb 22 23:25 Solar_Met_Daily_19980101_20210202_Nairobi.csv

./.config:
total 40
drwxr-xr-x 1 root root 4096 Feb 16 16:35 .
drwxr-xr-x 1 root root 4096 Feb 22 23:30 ..
-rw-r--r-- 1 root root    7 Feb 16 16:34 active_config
-rw-r--r-- 1 root root    0 Feb 16 16:35 config_sentinel
drwxr-xr-x 2 root root 4096 Feb 16 16:34 configurations
-rw------- 1 root root    5 Jan  1  2040 gce
-rw-r--r-- 1 root root    3 Feb 16 16:34 .last_opt_in_prompt.yaml
-rw-r--r-- 1 root root   37 Feb 16 16:34 .last_survey_prompt.yaml
-rw-r--r-- 1 root root  135 Feb 16 16:35 .last_update_check.jso

## [Pandas Profiling](https://pandas-profiling.github.io/pandas-profiling/docs/master/rtd/pages/introduction.html)

EDA using the pandas-profiling package

In [96]:
# First install and import required packages

# Pandas_Profiling requires a version of pandas that still suppors the join_axes parameter in the concat() method 
!pip install pandas==0.25.0

try:
    from pandas_profiling import ProfileReport
    import pandas_profiling as pd_pfl 
except ImportError as e:
    # module doesn't exist, deal with it.
    ! pip install pandas-profiling
    
def generateEDAreport_pandasProfile( p, sourceFileName ):
    print( "Generating Pandas Profiling EDA report for '%s'" % sourceFileName )
    print( "Dataframe rows %d" % len( p.index ) )
    # profile = ProfileReport( p, title='Pandas Profiling Report', html={'style':{'full_width':True}} )
    profile = ProfileReport( p )

    REPORT_DIRECTORY_PATH =os.path.join( os.path.split( sourceFileName )[ 0 ], "EDA_reports" )   # Currently not used
    # if not os.path.exists( REPORT_DIRECTORY_PATH ):
    # os.mkdir( REPORT_DIRECTORY_PATH )
    print( "Tentative destination report for reports '%s'" % REPORT_DIRECTORY_PATH)  
    
    # Saving results to a HTML file
    REPORT_SUFFIX ="_pandasProfl.html"
    OUTPUT_FILE = os.path.join( REPORT_DIRECTORY_PATH, os.path.split( sourceFileName )[ 1 ]+REPORT_SUFFIX )
    print( "Outputfile:", OUTPUT_FILE )
    # profile.to_file( OUTPUT_FILE )

    # Outputting results inline, as part of the current notebook
    profile.to_notebook_iframe()


## [Sweetviz](https://github.com/fbdesignpro/sweetviz)

EDA using the SweetViz package

In [19]:
# First install and import required packages
try:
    import pandas as pd
except ImportError as e:
    # module doesn't exist, deal with it.
    ! pip install pandas

try:
    import sweetviz as sv
except ImportError as e:
    # module doesn't exist, deal with it.
    ! pip install sweetviz

#EDA using Sweetviz
def generateEDAreport_sweetViz( p, sourceFileName ):
    print( "Generating SweetViz EDA report for '%s'" % sourceFileName )
    print( "Dataframe rows %d" % len( p.index ) )
    sweet_report = sv.analyze( p )
     
    # Saving results to HTML file
    
    # Invocation without any arguments will generate a "SWEETVIZ_REPORT.html" output file, without any prefix linking it to the source file.
    # sweet_report.show_html( )
    REPORT_DIRECTORY_PATH =os.path.join( os.path.split( sourceFileName )[ 0 ], "EDA_reports" )   # Currently not used
    if not os.path.exists( REPORT_DIRECTORY_PATH ):
        os.mkdir( REPORT_DIRECTORY_PATH )
    # print( "Tentative destination report for reports '%s'" % REPORT_DIRECTORY_PATH)  

    # Sending output to a specific destination directory other than working directory, however, requires invokation with explicit parameters 
    REPORT_SUFFIX ="_sweetViz.html"
    OUTPUT_FILE = os.path.join( REPORT_DIRECTORY_PATH, os.path.split( sourceFileName )[ 1 ]+REPORT_SUFFIX )
    # print( "Outputfile:", OUTPUT_FILE )
    
    # sweet_report.show_html( filepath=OUTPUT_FILE, open_browser=False, layout='widescreen', scale=None )
    sweet_report.show_html( filepath=OUTPUT_FILE, open_browser=False )
 
    # Outputting results inline, as part of the current notebook
    # sweet_report.show_notebook( )

## [Autoviz](https://github.com/AutoViML/AutoViz/blob/master/README.md)

EDA using the Autoviz package

References:

* [AutoViz: A New Tool for Automated Visualization, Dan Roth, Medium](https://towardsdatascience.com/autoviz-a-new-tool-for-automated-visualization-ec9c1744a6ad)  
_An XGBoost model is repeatedly used to determine the most consistent set of features determined to be important by using a random set of features each time; the most prominent selected features can then serve to guide future plotting and visualization. ... To do this effectively, AutoViz classifies the selected variables as categorical, numerical, boolean, NLP text and so on in order to understand how to best plot them._  
_Finally, using in-built heuristics, the tool will return the visuals deemed to have the greatest impact. AutoViz is also very much systematic: it uses all the selected variables with different chart types in order to deliver the best insights by letting the charts speak for themselves. ... AutoViz’ objective selection of features and plots can point data teams towards the best approaches using a systematic methodology and can greatly enhance a team’s productivity from the very outset of a project._  
_Check [notebook AutViz_test.ipynb](https://github.com/DanRothDataScience/autoviz_test/blob/master/AutoViz_test.ipynb) for an example notebook._


In [ ]:
# First install and import required packages
try:
    import pandas as pd
except ImportError as e:
    # module doesn't exist, deal with it.
    import pandas as pd

try:
    from autoviz.AutoViz_Class import AutoViz_Class
except ImportError as e:
    # module doesn't exist, deal with it.
    ! pip install autoviz

#EDA using Sweetviz
def generateEDAreport_AutoViz( p, sourceFileName, targetVariable ):
    print( "Generating AutoViz EDA report for '%s'" % sourceFileName )
    print( "Dataframe rows %d" % len( p.index ) )
    sweet_report = sv.analyze( p )
     
    # Saving results to HTML file is apparently not possible
    
    REPORT_DIRECTORY_PATH =os.path.join( os.path.split( sourceFileName )[ 0 ], "EDA_reports" )   # Currently not used
    if not os.path.exists( REPORT_DIRECTORY_PATH ):
        os.mkdir( REPORT_DIRECTORY_PATH )
    print( "Tentative destination report for reports '%s'" % REPORT_DIRECTORY_PATH)  

    # Sending output to a specific destination directory other than working directory, however, requires invokation with explicit parameters 
    REPORT_SUFFIX ="_AutoViz.html"
    OUTPUT_FILE = os.path.join( REPORT_DIRECTORY_PATH, os.path.split( sourceFileName )[ 1 ]+REPORT_SUFFIX )
    print( "Outputfile:", OUTPUT_FILE )
    
    # EDA using AutoViz

    # verbose option
    #     if 0, display minimal information but displays charts on your notebook
    #     if 1, print extra information on the notebook and also display charts
    #     if 2, you will not see any charts but they will be quietly generated and save in your local current directory under the AutoViz_Plots directory
    #           which will be created. Make sure you delete this folder periodically, otherwise, you will have lots of charts saved here if you used verbose=2 option a lot.
    # OUTPUT_FILE =INPUT_FILE+"/"+"AutoViz_Plots"
    # autoviz = AutoViz_Class().AutoViz( INPUT_FILE_PATH, verbose=0 )
    autoviz = AutoViz_Class().AutoViz( 
        filename='',
        sep=",",
        depVar=targetVariable,
        dfte= p,
        header=0,
        verbose=1,
        lowess=False,
        chart_format="svg",
        max_rows_analyzed=150000,
        max_cols_analyzed=30
    )

In [18]:
# New to both ways of manipulating file paths, os.path and libpath libs, so I'll initially write an os.path version
# and will develop its libpath equivalent on a separate cell 
from pathlib import Path
import os

# Loop recursively through directories, looking for relevant files type.
# DATA_ROOT_DIR = Path("C:/Users/alvar/OneDrive/Personal/Data")
DATA_ROOT_DIR = Path( "/content/drive/MyDrive/Data" )
# DATA_ROOT_DIR = DATA_ROOT_DIR / "Alvaro's Test Data"

if not DATA_ROOT_DIR.exists():
  print( "Root data directory does not exist:'%s'" % DATA_ROOT_DIR )


# Certain files to be excluded from search because of the impact that their layout has on pandas'
#  abilityto read table contents
exclusionList = [ 
      'electricity-domestic-consumption-data.xls',
      'renewable-in-electricity-production-share.xls'
]

# Recursive walk through files in DATA_ROOT_DIR
for path in DATA_ROOT_DIR.rglob("*"):
    # print( "Path:", os.fspath( path ) )
    if path.name in exclusionList:
        print( "File is to be explictly excluded from processing" )
        continue
    if path.suffix in [ '.csv' ]:
        print( "CSV file" )  
        p = pd.read_csv( os.fspath( path ) )
    elif path.suffix in [ '.xlsx', '.xlsm', '.xls' ]:
        print( "EXCEL file" )     
        p = pd.read_excel( os.fspath( path ) )
    else:
        # print( "File not among formats of interest" )
        continue
    p.head( 3 ) 
    generateEDAreport_sweetViz( p, os.fspath( path ) )



CSV file
Generating SweetViz EDA report for '/content/drive/MyDrive/Data/Alvaro's Test Data/Electricity_Accessibility_Areawise/WDI number-of-people-with-and-without-electricity-access.csv'
Dataframe rows 6958


                                             |          | [  0%]   00:00 -> (? left)

Tentative destination report for reports '/content/drive/MyDrive/Data/Alvaro's Test Data/Electricity_Accessibility_Areawise/EDA_reports'
Outputfile: /content/drive/MyDrive/Data/Alvaro's Test Data/Electricity_Accessibility_Areawise/EDA_reports/WDI number-of-people-with-and-without-electricity-access.csv_sweetViz.html
Report /content/drive/MyDrive/Data/Alvaro's Test Data/Electricity_Accessibility_Areawise/EDA_reports/WDI number-of-people-with-and-without-electricity-access.csv_sweetViz.html was generated.
-rw------- 1 root root 765920 Feb 23 02:43 "/content/drive/MyDrive/Data/Alvaro's Test Data/Electricity_Accessibility_Areawise/EDA_reports/WDI number-of-people-with-and-without-electricity-access.csv_sweetViz.html"
CSV file
Generating SweetViz EDA report for '/content/drive/MyDrive/Data/Alvaro's Test Data/Electricity_Accessibility_Areawise/OWID Data (people-without-electricity-country).csv'
Dataframe rows 6958


                                             |          | [  0%]   00:00 -> (? left)

Tentative destination report for reports '/content/drive/MyDrive/Data/Alvaro's Test Data/Electricity_Accessibility_Areawise/EDA_reports'
Outputfile: /content/drive/MyDrive/Data/Alvaro's Test Data/Electricity_Accessibility_Areawise/EDA_reports/OWID Data (people-without-electricity-country).csv_sweetViz.html
Report /content/drive/MyDrive/Data/Alvaro's Test Data/Electricity_Accessibility_Areawise/EDA_reports/OWID Data (people-without-electricity-country).csv_sweetViz.html was generated.
-rw------- 1 root root 673704 Feb 23 02:43 '/content/drive/MyDrive/Data/Alvaro'\''s Test Data/Electricity_Accessibility_Areawise/EDA_reports/OWID Data (people-without-electricity-country).csv_sweetViz.html'
File is to be explictly excluded from processing
EXCEL file
Generating SweetViz EDA report for '/content/drive/MyDrive/Data/Alvaro's Test Data/Electricity Production_using Renewable_energy/IRENA_RE_electricity_statistics_-_Query_tool.xlsm'
Dataframe rows 28


...

Tentative destination report for reports '/content/drive/MyDrive/Data/Alvaro's Test Data/Electricity Production/South Africa/EDA_reports'
Outputfile: /content/drive/MyDrive/Data/Alvaro's Test Data/Electricity Production/South Africa/EDA_reports/INT-Export-02-13-2021_11-19-00.csv_sweetViz.html
Report /content/drive/MyDrive/Data/Alvaro's Test Data/Electricity Production/South Africa/EDA_reports/INT-Export-02-13-2021_11-19-00.csv_sweetViz.html was generated.
-rw------- 1 root root 473852 Feb 23 02:45 "/content/drive/MyDrive/Data/Alvaro's Test Data/Electricity Production/South Africa/EDA_reports/INT-Export-02-13-2021_11-19-00.csv_sweetViz.html"
EXCEL file
Generating SweetViz EDA report for '/content/drive/MyDrive/Data/Alvaro's Test Data/Electricity Consumption/Enerdata_Energy_Statistical_Yearbook_2020.xlsx'
Dataframe rows 44


                                             |          | [  0%]   00:00 -> (? left)

Tentative destination report for reports '/content/drive/MyDrive/Data/Alvaro's Test Data/Electricity Consumption/EDA_reports'
Outputfile: /content/drive/MyDrive/Data/Alvaro's Test Data/Electricity Consumption/EDA_reports/Enerdata_Energy_Statistical_Yearbook_2020.xlsx_sweetViz.html
Report /content/drive/MyDrive/Data/Alvaro's Test Data/Electricity Consumption/EDA_reports/Enerdata_Energy_Statistical_Yearbook_2020.xlsx_sweetViz.html was generated.
-rw------- 1 root root 498465 Feb 23 02:45 "/content/drive/MyDrive/Data/Alvaro's Test Data/Electricity Consumption/EDA_reports/Enerdata_Energy_Statistical_Yearbook_2020.xlsx_sweetViz.html"
CSV file
Generating SweetViz EDA report for '/content/drive/MyDrive/Data/Alvaro's Test Data/Electricity Consumption/Countrywise electricity consumption(Our world Data source).csv'
Dataframe rows 6826


                                             |          | [  0%]   00:00 -> (? left)

Tentative destination report for reports '/content/drive/MyDrive/Data/Alvaro's Test Data/Electricity Consumption/EDA_reports'
Outputfile: /content/drive/MyDrive/Data/Alvaro's Test Data/Electricity Consumption/EDA_reports/Countrywise electricity consumption(Our world Data source).csv_sweetViz.html
Report /content/drive/MyDrive/Data/Alvaro's Test Data/Electricity Consumption/EDA_reports/Countrywise electricity consumption(Our world Data source).csv_sweetViz.html was generated.
-rw------- 1 root root 683854 Feb 23 02:45 '/content/drive/MyDrive/Data/Alvaro'\''s Test Data/Electricity Consumption/EDA_reports/Countrywise electricity consumption(Our world Data source).csv_sweetViz.html'
File is to be explictly excluded from processing
